## Codificación de las variables categoricas y escalado

**Elaborado por:** Jazmín Fernández Ramírez, Jacqueline Fernández Ramírez y Nina Odoux.
Máster Universitario de Ciencia de Datos (MUCD) - CUNEF Universidad.

En esta sección, nos enfocaremos en preparar nuestros datos para su uso en modelos de machine learning. Empezaremos con la codificación de las variables categóricas, transformando datos categóricos en un formato numérico utilizando técnicas como One Hot Encoding y TargetEncoding. Esto garantiza que las variables categóricas sean interpretadas adecuadamente por los modelos. Posteriormente, aplicaremos técnicas de escalado para normalizar los datos, asegurando que todas las características contribuyan de manera equitativa al entrenamiento del modelo. Finalmente, desarrollaremos y evaluaremos el rendimiento de un modelo de machine learning, utilizando algoritmos avanzados para detectar y predecir el fraude en cuentas bancarias. Este proceso integral es crucial para maximizar la precisión y eficacia de nuestras predicciones.

## Importación de librerías

Previamente se tuvo que instalar con el comando install la librería category_encoders

In [103]:
import pandas as pd 
import numpy as np
import sklearn
from sklearn.pipeline import Pipeline
from sklearn import metrics
import category_encoders as ce
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import GradientBoostingClassifier
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.preprocessing import StandardScaler
from category_encoders import TargetEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, \
                            silhouette_score, recall_score, precision_score, make_scorer, \
                            roc_auc_score, f1_score, precision_recall_curve, accuracy_score, roc_auc_score, \
                            classification_report, confusion_matrix
from sklearn.compose import ColumnTransformer
from sklearn import metrics
from sklearn.metrics import plot_confusion_matrix
seed = np.random.seed(123)

In [64]:
df_application_train = pd.read_csv("../data/preprocessed_data/train_preprocessed_missing_outlier.csv")
df_application_test = pd.read_csv("../data/preprocessed_data/test_preprocessed_missing_outlier.csv")

# Mostrar las primeras filas del DataFrame para verificación
print("Primeras filas del conjunto de entrenamiento:")
print(df_application_train.head())

print("Primeras filas del conjunto de prueba:")
print(df_application_test.head())

Primeras filas del conjunto de entrenamiento:
   COMMONAREA_MEDI  COMMONAREA_AVG  COMMONAREA_MODE  NONLIVINGAPARTMENTS_MODE  \
0           0.1366          0.1358            0.137                    0.0039   
1         -99.0000        -99.0000          -99.000                  -99.0000   
2           0.0000          0.0000            0.000                    0.0039   
3         -99.0000        -99.0000          -99.000                  -99.0000   
4         -99.0000        -99.0000          -99.000                  -99.0000   

   NONLIVINGAPARTMENTS_AVG  NONLIVINGAPARTMENTS_MEDI     FONDKAPREMONT_MODE  \
0                   0.0039                    0.0039       reg oper account   
1                 -99.0000                  -99.0000              SIN VALOR   
2                   0.0039                    0.0039  reg oper spec account   
3                 -99.0000                  -99.0000              SIN VALOR   
4                 -99.0000                  -99.0000       reg oper acco

In [65]:
df_application_train.columns

Index(['COMMONAREA_MEDI', 'COMMONAREA_AVG', 'COMMONAREA_MODE',
       'NONLIVINGAPARTMENTS_MODE', 'NONLIVINGAPARTMENTS_AVG',
       'NONLIVINGAPARTMENTS_MEDI', 'FONDKAPREMONT_MODE',
       'LIVINGAPARTMENTS_MODE', 'LIVINGAPARTMENTS_AVG',
       'LIVINGAPARTMENTS_MEDI',
       ...
       'DAYS_REGISTRATION', 'DAYS_EMPLOYED', 'DAYS_BIRTH',
       'REGION_POPULATION_RELATIVE', 'NAME_HOUSING_TYPE', 'NAME_FAMILY_STATUS',
       'NAME_EDUCATION_TYPE', 'NAME_INCOME_TYPE', 'SK_ID_CURR', 'TARGET'],
      dtype='object', length=122)

In [66]:
df_application_train.dtypes

COMMONAREA_MEDI             float64
COMMONAREA_AVG              float64
COMMONAREA_MODE             float64
NONLIVINGAPARTMENTS_MODE    float64
NONLIVINGAPARTMENTS_AVG     float64
                             ...   
NAME_FAMILY_STATUS           object
NAME_EDUCATION_TYPE          object
NAME_INCOME_TYPE             object
SK_ID_CURR                    int64
TARGET                        int64
Length: 122, dtype: object

## Codificación de la variable objetivo

* 1: Cliente con pagos retrasados en las primeras cuotas del préstamo.

* 0: Cliente sin pagos retrasados en las primeras cuotas del préstamo.

In [68]:
df_application_train['TARGET'].value_counts()

0    226148
1     19860
Name: TARGET, dtype: int64

In [69]:
# Asegurar que la columna 'TARGET' está tipificada como entero
df_application_train['TARGET'] = df_application_train['TARGET'].astype(int)
df_application_test['TARGET'] = df_application_test['TARGET'].astype(int)

# Verificar los resultados
print(df_application_train['TARGET'].dtype)
print(df_application_test['TARGET'].dtype)

# Mostrar la distribución de valores en 'TARGET'
print(df_application_train['TARGET'].value_counts())
print(df_application_test['TARGET'].value_counts())

int32
int32
0    226148
1     19860
Name: TARGET, dtype: int64
0    56538
1     4965
Name: TARGET, dtype: int64


## Verificación

Este código también incluye una verificación para asegurarnos de que la columna TARGET está correctamente tipificada y muestra la distribución de valores en la columna TARGET.

In [71]:
X_train = df_application_train.drop('TARGET',axis=1)
X_test = df_application_test.drop('TARGET',axis=1)
y_train = df_application_train['TARGET']
y_test = df_application_test['TARGET']

## Codificación del resto de variables categóricas



En esta fase, vamos a transformar las variables categóricas de nuestro conjunto de datos en números que los modelos de machine learning puedan entender. Utilizaremos diferentes técnicas según el tipo de variable. Para las variables sin un orden específico, aplicaremos One Hot Encoding, lo que significa que cada categoría se convertirá en una columna separada con valores binarios (0 o 1). Esto ayuda a que los modelos traten cada categoría de manera independiente y justa.

Para algunas variables clave, usaremos Target Encoding, que asigna un valor numérico basado en su relación con la variable objetivo. Este enfoque permite capturar la importancia de ciertas categorías en la predicción del fraude en cuentas bancarias, maximizando así la precisión de nuestros modelos. Al combinar estos métodos, nos aseguramos de que los datos estén bien preparados y listos para entrenar modelos efectivos y fiables.

In [74]:
list_columns_cat = list(X_train.select_dtypes("object", "category").columns)
list_other = list(set(X_train.columns)-set(list_columns_cat))

print(list_columns_cat)
print(list_other)

['FONDKAPREMONT_MODE', 'WALLSMATERIAL_MODE', 'HOUSETYPE_MODE', 'EMERGENCYSTATE_MODE', 'OCCUPATION_TYPE', 'NAME_TYPE_SUITE', 'NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'WEEKDAY_APPR_PROCESS_START', 'ORGANIZATION_TYPE', 'NAME_HOUSING_TYPE', 'NAME_FAMILY_STATUS', 'NAME_EDUCATION_TYPE', 'NAME_INCOME_TYPE']
['DAYS_REGISTRATION', 'SK_ID_CURR', 'DAYS_EMPLOYED', 'NONLIVINGAPARTMENTS_AVG', 'ELEVATORS_AVG', 'AMT_REQ_CREDIT_BUREAU_MON', 'FLAG_DOCUMENT_15', 'FLAG_CONT_MOBILE', 'YEARS_BUILD_MODE', 'FLAG_DOCUMENT_14', 'REGION_RATING_CLIENT', 'FLAG_DOCUMENT_12', 'FLAG_DOCUMENT_18', 'DAYS_LAST_PHONE_CHANGE', 'FLAG_DOCUMENT_5', 'LIVE_REGION_NOT_WORK_REGION', 'REG_CITY_NOT_WORK_CITY', 'AMT_REQ_CREDIT_BUREAU_HOUR', 'REG_REGION_NOT_WORK_REGION', 'APARTMENTS_MEDI', 'AMT_REQ_CREDIT_BUREAU_DAY', 'BASEMENTAREA_MODE', 'LANDAREA_MODE', 'ENTRANCES_AVG', 'REG_REGION_NOT_LIVE_REGION', 'AMT_REQ_CREDIT_BUREAU_YEAR', 'FLAG_PHONE', 'AMT_GOODS_PRICE', 'BASEMENTAREA_AVG', 'NONLIVINGAREA_MODE

## Verificar el número de categorías

En primera instancia, se crea una función que cuente el número de categorías en cada una de las variables categóricas.

In [85]:
# Verificar el número de categorías en cada variable categórica
def count_categories(df, cat_columns):
    cat_counts = {}
    for col in cat_columns:
        cat_counts[col] = df[col].nunique()
    return pd.Series(cat_counts).sort_values(ascending=False)

# Contar las categorías en las variables categóricas de X_train
category_counts = count_categories(X_train, list_columns_cat)

# Mostrar el número de categorías en cada variable categórica
print("Número de categorías en cada variable categórica:")
print(category_counts)

Número de categorías en cada variable categórica:
ORGANIZATION_TYPE             58
OCCUPATION_TYPE               19
WALLSMATERIAL_MODE             8
NAME_TYPE_SUITE                8
NAME_INCOME_TYPE               8
WEEKDAY_APPR_PROCESS_START     7
NAME_HOUSING_TYPE              6
NAME_FAMILY_STATUS             6
FONDKAPREMONT_MODE             5
NAME_EDUCATION_TYPE            5
HOUSETYPE_MODE                 4
EMERGENCYSTATE_MODE            3
CODE_GENDER                    3
NAME_CONTRACT_TYPE             2
FLAG_OWN_CAR                   2
FLAG_OWN_REALTY                2
dtype: int64


## Decidir tipo de codificación

Con base en el número de categorías, se puede seleccionar cuál método de codificación emplear, como se detalla a continuación,

* One-Hot Encoding: Apto para variables con un número bajo de categorías.
* Target Encoding: Apto para variables con un número alto de categorías para evitar la creación de demasiadas columnas.

In [88]:
# Definir umbrales para decidir el tipo de codificación
threshold_one_hot = 10  # Puedes ajustar este valor según tus necesidades

# Variables para One-Hot Encoding y Target Encoding
one_hot_columns = category_counts[category_counts <= threshold_one_hot].index.tolist()
target_encoding_columns = category_counts[category_counts > threshold_one_hot].index.tolist()

print("Variables para One-Hot Encoding:", one_hot_columns)
print("Variables para Target Encoding:", target_encoding_columns)

Variables para One-Hot Encoding: ['WALLSMATERIAL_MODE', 'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'WEEKDAY_APPR_PROCESS_START', 'NAME_HOUSING_TYPE', 'NAME_FAMILY_STATUS', 'FONDKAPREMONT_MODE', 'NAME_EDUCATION_TYPE', 'HOUSETYPE_MODE', 'EMERGENCYSTATE_MODE', 'CODE_GENDER', 'NAME_CONTRACT_TYPE', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY']
Variables para Target Encoding: ['ORGANIZATION_TYPE', 'OCCUPATION_TYPE']


In [90]:
# Lista de variables para One-Hot Encoding
one_hot_columns = ['WALLSMATERIAL_MODE', 'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'WEEKDAY_APPR_PROCESS_START', 
                   'NAME_HOUSING_TYPE', 'NAME_FAMILY_STATUS', 'FONDKAPREMONT_MODE', 'NAME_EDUCATION_TYPE', 
                   'HOUSETYPE_MODE', 'EMERGENCYSTATE_MODE', 'CODE_GENDER', 'NAME_CONTRACT_TYPE', 'FLAG_OWN_CAR', 
                   'FLAG_OWN_REALTY']

# Aplicar One-Hot Encoding
one_hot_encoder = OneHotEncoder(sparse=False, drop='first')
one_hot_encoded = one_hot_encoder.fit_transform(X_train[one_hot_columns])

# Crear un DataFrame con las variables One-Hot Encoded
one_hot_encoded_df = pd.DataFrame(one_hot_encoded, columns=one_hot_encoder.get_feature_names(one_hot_columns))

# Combinar el DataFrame original con las variables One-Hot Encoded
X_train = X_train.drop(one_hot_columns, axis=1)
X_train = pd.concat([X_train.reset_index(drop=True), one_hot_encoded_df.reset_index(drop=True)], axis=1)

# Lista de variables para Target Encoding
target_encoding_columns = ['ORGANIZATION_TYPE', 'OCCUPATION_TYPE']

# Aplicar Target Encoding
target_encoder = TargetEncoder(cols=target_encoding_columns)
X_train[target_encoding_columns] = target_encoder.fit_transform(X_train[target_encoding_columns], y_train)

# Verificar las primeras filas del DataFrame después de One-Hot Encoding y Target Encoding
print("Primeras filas del DataFrame después de One-Hot Encoding y Target Encoding:")
print(X_train.head())

# Verificar las dimensiones del DataFrame
print("Dimensiones del DataFrame:")
print(X_train.shape)

Primeras filas del DataFrame después de One-Hot Encoding y Target Encoding:
   COMMONAREA_MEDI  COMMONAREA_AVG  COMMONAREA_MODE  NONLIVINGAPARTMENTS_MODE  \
0           0.1366          0.1358            0.137                    0.0039   
1         -99.0000        -99.0000          -99.000                  -99.0000   
2           0.0000          0.0000            0.000                    0.0039   
3         -99.0000        -99.0000          -99.000                  -99.0000   
4         -99.0000        -99.0000          -99.000                  -99.0000   

   NONLIVINGAPARTMENTS_AVG  NONLIVINGAPARTMENTS_MEDI  LIVINGAPARTMENTS_MODE  \
0                   0.0039                    0.0039                 0.1983   
1                 -99.0000                  -99.0000               -99.0000   
2                   0.0039                    0.0039                 0.0367   
3                 -99.0000                  -99.0000               -99.0000   
4                 -99.0000                

In [92]:
X_train.dtypes.to_dict()

{'COMMONAREA_MEDI': dtype('float64'),
 'COMMONAREA_AVG': dtype('float64'),
 'COMMONAREA_MODE': dtype('float64'),
 'NONLIVINGAPARTMENTS_MODE': dtype('float64'),
 'NONLIVINGAPARTMENTS_AVG': dtype('float64'),
 'NONLIVINGAPARTMENTS_MEDI': dtype('float64'),
 'LIVINGAPARTMENTS_MODE': dtype('float64'),
 'LIVINGAPARTMENTS_AVG': dtype('float64'),
 'LIVINGAPARTMENTS_MEDI': dtype('float64'),
 'FLOORSMIN_AVG': dtype('float64'),
 'FLOORSMIN_MODE': dtype('float64'),
 'FLOORSMIN_MEDI': dtype('float64'),
 'YEARS_BUILD_MEDI': dtype('float64'),
 'YEARS_BUILD_MODE': dtype('float64'),
 'YEARS_BUILD_AVG': dtype('float64'),
 'OWN_CAR_AGE': dtype('float64'),
 'LANDAREA_MEDI': dtype('float64'),
 'LANDAREA_MODE': dtype('float64'),
 'LANDAREA_AVG': dtype('float64'),
 'BASEMENTAREA_MEDI': dtype('float64'),
 'BASEMENTAREA_AVG': dtype('float64'),
 'BASEMENTAREA_MODE': dtype('float64'),
 'EXT_SOURCE_1': dtype('float64'),
 'NONLIVINGAREA_MODE': dtype('float64'),
 'NONLIVINGAREA_AVG': dtype('float64'),
 'NONLIVINGARE

## Justificación

* **Variables Codificadas con One-Hot Encoding:**

En nuestro DataFrame, algunas variables categóricas han sido codificadas utilizando One-Hot Encoding. Este proceso ha transformado cada categoría de variables como 'WALLSMATERIAL_MODE', 'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', entre otras, en nuevas columnas que representan cada categoría individualmente. Estas nuevas columnas tienen sufijos que indican la categoría correspondiente y están tipificadas como float64, lo que confirma que la codificación se ha realizado correctamente. Por ejemplo, ahora encontramos columnas como 'WALLSMATERIAL_MODE_Mixed', 'NAME_TYPE_SUITE_Family', y 'NAME_INCOME_TYPE_Commercial associate', entre otras.

* **Variables Codificadas con Target Encoding:**

Para otras variables, como 'ORGANIZATION_TYPE' y 'OCCUPATION_TYPE', hemos aplicado Target Encoding. Este método ha reemplazado cada categoría con el promedio de la variable objetivo asociada, asegurando que las nuevas columnas sean de tipo float64. Así, cada categoría se convierte en un valor que refleja su relación con la variable objetivo, facilitando un análisis más preciso y significativo.

## Escalado de las variables

El escalado de las variables es un paso fundamental en el preprocesamiento de datos para modelos de machine learning. Este proceso garantiza que todas las características contribuyan de manera equitativa durante el entrenamiento del modelo, evitando que algunas variables dominen sobre otras debido a diferencias en la escala de sus valores. Utilizaremos técnicas de escalado proporcionadas por Scikit-learn, una biblioteca ampliamente reconocida en el ámbito del aprendizaje automático. Estas técnicas ajustarán las características a una escala común, mejorando así la precisión y la eficiencia de nuestros algoritmos predictivos. A continuación, se presenta el código de escalado que se implementará:

In [60]:
print("Columnas disponibles en X_train:")
print(X_train.columns.tolist())

Columnas disponibles en X_train:
['COMMONAREA_MEDI', 'COMMONAREA_AVG', 'COMMONAREA_MODE', 'NONLIVINGAPARTMENTS_MODE', 'NONLIVINGAPARTMENTS_AVG', 'NONLIVINGAPARTMENTS_MEDI', 'LIVINGAPARTMENTS_MODE', 'LIVINGAPARTMENTS_AVG', 'LIVINGAPARTMENTS_MEDI', 'FLOORSMIN_AVG', 'FLOORSMIN_MODE', 'FLOORSMIN_MEDI', 'YEARS_BUILD_MEDI', 'YEARS_BUILD_MODE', 'YEARS_BUILD_AVG', 'OWN_CAR_AGE', 'LANDAREA_MEDI', 'LANDAREA_MODE', 'LANDAREA_AVG', 'BASEMENTAREA_MEDI', 'BASEMENTAREA_AVG', 'BASEMENTAREA_MODE', 'EXT_SOURCE_1', 'NONLIVINGAREA_MODE', 'NONLIVINGAREA_AVG', 'NONLIVINGAREA_MEDI', 'ELEVATORS_MEDI', 'ELEVATORS_AVG', 'ELEVATORS_MODE', 'APARTMENTS_MEDI', 'APARTMENTS_AVG', 'APARTMENTS_MODE', 'ENTRANCES_MEDI', 'ENTRANCES_AVG', 'ENTRANCES_MODE', 'LIVINGAREA_AVG', 'LIVINGAREA_MODE', 'LIVINGAREA_MEDI', 'FLOORSMAX_MODE', 'FLOORSMAX_MEDI', 'FLOORSMAX_AVG', 'YEARS_BEGINEXPLUATATION_MODE', 'YEARS_BEGINEXPLUATATION_MEDI', 'YEARS_BEGINEXPLUATATION_AVG', 'TOTALAREA_MODE', 'OCCUPATION_TYPE', 'EXT_SOURCE_3', 'AMT_REQ_CREDI

In [107]:
# X_train y X_test son DataFrames codificados y y_train es variable objetivo

# Lista de variables para One-Hot Encoding y Target Encoding (ya codificadas)
one_hot_columns = ['WALLSMATERIAL_MODE', 'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'WEEKDAY_APPR_PROCESS_START', 
                   'NAME_HOUSING_TYPE', 'NAME_FAMILY_STATUS', 'FONDKAPREMONT_MODE', 'NAME_EDUCATION_TYPE', 
                   'HOUSETYPE_MODE', 'EMERGENCYSTATE_MODE', 'CODE_GENDER', 'NAME_CONTRACT_TYPE', 'FLAG_OWN_CAR', 
                   'FLAG_OWN_REALTY']

target_encoding_columns = ['ORGANIZATION_TYPE', 'OCCUPATION_TYPE']

# Identificar columnas numéricas
num_vars = X_train.select_dtypes(include=['float64', 'int64']).columns.difference(one_hot_columns + target_encoding_columns).tolist()

# Alinear las columnas de X_train y X_test para asegurarse de que tengan el mismo número de características
X_train_aligned, X_test_aligned = X_train.align(X_test, join='outer', axis=1, fill_value=0)

# Crear una pipeline para el escalado de las variables numéricas
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_vars)  # Escalado de variables numéricas
    ],
    remainder='passthrough'  # Mantener otras columnas sin cambiar
)

# Ajustar el preprocesador en el conjunto de entrenamiento y transformar los datos de entrenamiento y prueba
X_train_scaled = preprocessor.fit_transform(X_train_aligned, y_train)
X_test_scaled = preprocessor.transform(X_test_aligned)

# Convertir los resultados a DataFrames
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train_aligned.columns, index=X_train.index)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test_aligned.columns, index=X_test.index)

# Verificar los resultados
print("Conjunto de entrenamiento escalado:")
print(X_train_scaled.head())

print("Conjunto de prueba escalado:")
print(X_test_scaled.head())

Conjunto de entrenamiento escalado:
   AMT_ANNUITY  AMT_CREDIT  AMT_GOODS_PRICE  AMT_INCOME_TOTAL  \
0    -0.590344   -0.802005        -0.810181         -0.219251   
1     0.684449    1.650905         1.529784          0.044305   
2     0.373033   -0.251782        -0.237377         -0.043547   
3    -0.758911   -0.943833        -1.005178          0.044305   
4     0.503796    0.189605         0.371989         -0.043547   

   AMT_REQ_CREDIT_BUREAU_DAY  AMT_REQ_CREDIT_BUREAU_HOUR  \
0                   0.394815                    0.394831   
1                   0.394815                    0.394831   
2                   0.394815                    0.394831   
3                  -2.531498                   -2.531503   
4                   0.394815                    0.394831   

   AMT_REQ_CREDIT_BUREAU_MON  AMT_REQ_CREDIT_BUREAU_QRT  \
0                   0.387025                   0.387070   
1                   0.387025                   0.387070   
2                   0.387025       

El resultado muestra que las variables numéricas han sido correctamente escaladas y las columnas están adecuadamente alineadas en los conjuntos de entrenamiento y prueba. Esto es un buen indicio de que el proceso de preprocesamiento ha sido exitoso y los datos están listos para ser utilizados en el modelado.

### Escalado de variables numéricas:

* Variables como AMT_ANNUITY, AMT_CREDIT, AMT_GOODS_PRICE, AMT_INCOME_TOTAL, etc., se han escalado, lo que se refleja en los valores transformados (por ejemplo, los valores ahora están centrados en torno a cero y una desviación estándar de 1).

### Consistencia entre los conjuntos de datos:

* Las columnas y sus valores en ambos conjuntos de datos (X_train y X_test) están correctamente alineados, lo que garantiza que los modelos de machine learning puedan entrenarse y evaluarse sin problemas de desajuste de columnas.

### Protección de principios de ciencia de datos y aprendizaje automático:

* Se han respetado los principios al tratar adecuadamente las variables categóricas con One-Hot Encoding y Target Encoding, y al escalar solo las variables numéricas.

In [109]:
# Verificar la media y desviación estándar
print("Estadísticas del conjunto de entrenamiento escalado:")
print(X_train_scaled.describe())

print("Estadísticas del conjunto de prueba escalado:")
print(X_test_scaled.describe())

Estadísticas del conjunto de entrenamiento escalado:
        AMT_ANNUITY    AMT_CREDIT  AMT_GOODS_PRICE  AMT_INCOME_TOTAL  \
count  2.460080e+05  2.460080e+05     2.460080e+05      2.460080e+05   
mean  -1.205139e-16 -2.802365e-17     1.195030e-17     -5.375832e-18   
std    1.000002e+00  1.000002e+00     1.000002e+00      1.000002e+00   
min   -1.873193e+00 -1.377185e+00    -1.456377e+00     -5.583601e-01   
25%   -7.285445e-01 -8.176174e-01    -8.101810e-01     -2.192513e-01   
50%   -1.515722e-01 -2.119634e-01    -2.373771e-01     -9.625849e-02   
75%    5.158813e-01  5.219882e-01     3.841761e-01      1.321568e-01   
max    1.590109e+01  8.583124e+00     9.512477e+00      4.561720e+02   

       AMT_REQ_CREDIT_BUREAU_DAY  AMT_REQ_CREDIT_BUREAU_HOUR  \
count               2.460080e+05                2.460080e+05   
mean                1.643438e-17                9.127000e-18   
std                 1.000002e+00                1.000002e+00   
min                -2.531498e+00          

# Conclusiones finales de la EDA disponibles en la carpeta ''reports''

# GUARDADO DE TABLAS

In [113]:
# Guardar los DataFrames preprocesados en archivos CSV
X_train_scaled.to_csv("../data/preprocessed_data/X_train_preprocessing_scaled.csv", index=False)
X_test_scaled.to_csv("../data/preprocessed_data/X_test_preprocessing_scaled.csv", index=False)
# Guardar y_train y y_test

y_train.to_csv("../data/preprocessed_data/y_train_preprocessing_scaled.csv", index=False)
y_test.to_csv("../data/preprocessed_data/y_test_preprocessing_scaled.csv", index=False)

# Verificar los resultados de la operación
print("Archivos CSV creados y guardados con éxito.")


Archivos CSV creados y guardados con éxito.
